In [8]:
import importlib.util

In [9]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt
from datetime import datetime

In [10]:
spec = importlib.util.spec_from_file_location("PandasRepo", "/Users/dennis/PycharmProjects/IbTrader/PandasRepo.py")
PandasRepoModule = importlib.util.module_from_spec(spec)
spec.loader.exec_module(PandasRepoModule)

In [11]:
class FirstBacktraderStrategy(bt.Strategy):

    def __init__(self):
        self.rsi = dict()
        for symbol in self.getdatanames():
            self.rsi[symbol] = bt.indicators.RSI_SMA(self.getdatabyname(symbol).close, period=21)

    def next(self):
        for symbol in self.getdatanames():
            pos = self.getpositionbyname(symbol).size or 0
            if pos == 0:
                if self.rsi[symbol][0] < 30:
                    self.buy(data=self.getdatabyname(symbol), size=100)
            else:
                if self.rsi[symbol][0] > 70:
                    self.sell(data=self.getdatabyname(symbol), size=100)

In [42]:
class BacktraderWrapper:
    def __init__(self,
                 startCash = 10000,
                 universe = ["IYM", "IYC", "IYK", "IYE", "IYF", "IYH", "IYR", "IYW", "IDU"],
                 startDate = datetime(2016, 1, 1),
                 endDate = datetime(2016, 12, 31)):
        self._startCash = startCash
        self._universe = universe
        self._startDate = startDate
        self._endDate = endDate

    def RunBackTest(self):
        # Create an instance of cerebro
        cerebro = bt.Cerebro()

        # Add our strategy
        cerebro.addstrategy(FirstBacktraderStrategy)

        # Add data feed
        for symbol in self._universe:
            df = PandasRepoModule.PandasRepo().GetData(symbol, self._startDate, self._endDate)
            data = bt.feeds.PandasData(dataname=df)
            cerebro.adddata(data, name=symbol)

        # Set our desired cash start
        cerebro.broker.setcash(self._startCash)

        # Set up pyfolio
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

        # Run over everything
        results = cerebro.run()
        strat = results[0]
        pyfoliozer = strat.analyzers.getbyname('pyfolio')
        returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
        print(returns.head())
        print(returns.index[0].tz)
        
        benchmarkDf = df = PandasRepoModule.PandasRepo().GetData('SPY', self._startDate, self._endDate)
        benchmarkDf = benchmarkDf['Close'].pct_change().fillna(0).tz_localize(tz='UTC')
        print(benchmarkDf.head())
        print(benchmarkDf.index[0].tz)

        import pyfolio as pf
        pf.create_full_tear_sheet(
            returns,
            positions=positions,
            benchmark_rets = benchmarkDf,
            transactions=transactions,
            gross_lev=gross_lev,
            live_start_date='2016-01-01',  # This date is sample specific
            round_trips=True)

In [43]:
BacktraderWrapper(startCash=100000).RunBackTest()

index
2016-01-01 00:00:00+00:00    0.0
2016-01-04 00:00:00+00:00    0.0
2016-01-05 00:00:00+00:00    0.0
2016-01-06 00:00:00+00:00    0.0
2016-01-07 00:00:00+00:00    0.0
Name: return, dtype: float64
UTC
2016-01-01 00:00:00+00:00    0.000000
2016-01-04 00:00:00+00:00   -0.013983
2016-01-05 00:00:00+00:00    0.001695
2016-01-06 00:00:00+00:00   -0.012614
2016-01-07 00:00:00+00:00   -0.023992
Name: Close, dtype: float64
UTC
Entire data start date: 2016-01-01
Entire data end date: 2017-12-29


Out-of-Sample Months: 24
Backtest Months: 0


/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/pyfolio/timeseries.py:416: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  beta, alpha = sp.stats.linregress(factor_returns.loc[ret_index].values,


ValueError: Inputs must not be empty.